In [2]:
c_l = ["F3048","D0218","B0507","B8987","F5029","D0228","F4981","D8132","F2543","F4052","E0588","F2768","B8340","F2590","A0032","E0498","C4825","F3204","A9831","A1237","D3495","A2906","F4209","F2638","F4936","F5566","F5238","C6083","B8815","C4121","F5289","A7386","F4945","B8082","D3381","D9582","C9117","F5134","F4670","A3781","B8412","E1967","B9831","E7763","A2252","A4727","A7909","A6116","B8488","A2834","F3570","B7283","A0365","A9326","D3501","B7175","C4349","D0158","B8846","F4128","E8154","E8717","A0284","A7641","A6933","B2832","F5155","A0502","D0976","F2013","F5230","F5369","C2354","F4366","F1406","D6358","D0095","B9438","A3224","B4177","E5975","A0256","C3025","F2291","F4201","D2415","F4624","F3691","C6513","F3884","F3742","E5090","C7753","A1151","F5187","F5067","D2192","B7900","B7270","A2609","E6692","C8702","F2554","C6474","F3988","B8010","C8898","F4642","F1682","A2171","C6392","A2777","D4421","B8806","E8115","A0114","A8445","A5087","A5835","D6788","E9364","F3322","A1781","F1636","B7619","B3938","A1664","A0190","F4939","B1161","F4950","C6494","F3754","E7524","E8414","F2076","C5031","F5305","F5199","A0508","D5238","C7944","D8442","E7779","E0851","D3601","C4757","A2762","C0192","F0129","C4641","D9542","C0463","D2304","F4590","C0287","C1844","F5172","F2930","F2987","F3378","B8479","B7105","F2970","B6705","A2820","A1270","D8067","E5774","E6552","E5212","D3130","D9049","A0933","E7518","A0386","D8347","D5205","D8715","F4627","B8890","D3475","F0927","E2027","F4734","D1597","D3948","D7612","D1897","D5650","E6089","E2213","F1150","A2440","A4316","A2447","A1318","A8796","E1201","A2556","C5787","B3324","B7168","F3866","B2844","E6012","B5501","B1018","F2494","F3253","F1447","D3917","E5626","E5020","E1333","C8417","D5856","F2560","A2197","B8540","C0384","A1499","E8955","E7650","F1707","C3066","F1163","D1622","F3749","C9688","F5160","F5143","C3268","F0156","C6322","F0412","F5097","E1116","C8449","F5583","F4653","F4182","F3914","F5158","F0400","D0230","C7849","B8475","E9622","C0548","B7831","C1351","A4388","B2855","B3940","B2706","B1016","B7472","F3181","E5418","F5031","B6274","D3634","A0570","B4808","D7541","C0455","C8110","F4257","A0493","F5059","D0546","D9669","C5538","D6545","E5823","D7843","F1650","D0177","D7535","C5581","A1382","E9952","D9029","C9687","A2363","F4080","C0968","D6614","C0812","F4660","C6209","D1442","E5399","E3386","F1590","E9664","E8081","F1658","F2248","A2442","D2115","A6279","A7768","A1018","A1092","B6194","A8434","B8450","B2243","B6247","F3486","B2150","B8137","C5283","E9523","B6633","B8334","B6479","B5118","B8108","B8466","E0107","E3401","D8207","F2277","D0547","F5203","D9743","C3040","F4556","C2179","D6994","C5526","C7448","F5104","E4495","C4437","F5406","C5323","D0967","B9093","E2233","C5844","F2052","D2581","D4013","F1453","C5378","E9817","C7569","D3658","C6136","C5608","F4561","E0877","C5867","E7171","E5274","F1780","C1162","F4816","B9445","D0466","C3095","D4597","A4407","A2523","B0779","A7305","B0121","B4783","B7103","B3509","B6918","B8152","F4935","B6340","B8474","B6172","F4473","D9548","B8350","B5861","E7167","C1381","A1048","C1166","F3832","A1626","F0088","A0968","C1733","E0760","E0490","B3446","D5991","D5224","E6691","F5412","D1652","D2637","C2935","D7388","D5659","F4256","C9455","C5305","F1209","D2024","D9257","F3791","F4073","E4826","F0226","E8296","E3070","E0936","C0643","D1686","C6054","F0788","C2060","E1166","C3076","E8397","E0507","F5396","E9703","F4322","C1717","C3462","B9658","E8814","C1774","C1079","C6388","F1244","F0518","D0148","C6319","C3045","A0482","E2256","F5089","E9528","C9692","F0492","A4286","A5337","A5388","A5549","A5856","A7014","A8297","F3174","A0499","B5576","A6139","F5339","A6073","B2252","B0722","B3416","B7106","F3217","C5671","A9765","F3159","D2990","B0843","C1103","A1245","C8078","E8333","B0468","E3906","A0379","F5229","B8382","E7744","C0601","D3888","C6738","D0863","C0753","C6504","F1366","F0485","C2699","D5666","D1008","D9203","D6962","E1664","A2725","F4539","D7406","D5261","B9913","A2002","F2609","C8739","A1604","F5176","F2014","E4319","E0579","C3903","A3113","D0817","C0486","E7757","D7169","C4590","A1871","D9334","A1627","E4029","C2347","C8947","D4162","C8836","F3944","E7697","F3148","B9252","A0186","F3955","B9552","D1696","F1059","C5717","F4559","D8110","C2244","E3380","F1297","F3605","D0768","A1206","C0180","E1698","E3435","A3829","A4319","A4168","A4882","F2788","A5569","B3133","B6298","B0953","B7814","B7296","A3321","A8472","B6246","B4489","F3637","B7742","B0077","B0849","B2512","A0357","A2892","C1154","B8843","E8820","B7589","B0947","B4924","D0533","C6986","F2550","F3523","F5119","B3523","E0620","B8361","D5670","B6384","E7090","C0342","C0743","E0340","F4351","F1839","E7204","F3539","A2163","E5578","C3291","C1798","F0124","F3751","E7248","D2363","C0578","F1557","C5669","F5064","D2606","E9856","D1166","B8489","F5082","C6820","F5076","C5558","D4540","D7331","F4018","A1954","E4763","F4664","D1414","B9639","F0582","D1131","D3756","E2449","D1595","E2900","C6125","C4672","D2877","B9852","D8546","F4414","E7195","A1303","D2615","C8497","D3608","F4770","D0610","D4408","D0057","F3758","C7310","E2389","C3719","F1057","F4567","C6884","C0258","C6694","D7857","E9707","A3376","C2849","F0733","D8368","D8722","E8855","E7026","D0315","E7192","D1407","B8932","D3958","F2174","C3266","D0999","F5447","E8230","D7378","E9125","A3714","A3741","A3892","A4061","A4251","A4418","F2809","A5090","A7354","A6927","C2397","A8586","A8831","A7345","A6255","A8874","B2258","A0308","A9706","B4196","B8430","B0616","B0037","F3813","B0206","A9628","D4817","F1226","C1090","A2703","B0488","C2489","B1116","B1773","D3428","E0813","B4397","C8118","E9656","D7496","C1688","D1210","E9831","C5580","B3093","C3059","D8320","A2166","F0410","E4174","E8249","C9872","E0336","D6638","B1088","D3396","D0757","F1936","D0032","D1431","D2169","C5131","C6785","B8523","F5265","D6533","C7840","E5758","F4525","B9364","D5811","F0891","C2251","C5822","C5263","C8541","E8246","C0357","D0887","C6421","D8066","C3929","E0209","A6049","E0599","C7548","E5676","D0394","D9557","D1182","E2227","D9533","D1483","C5899","C5944","D8828","C6103","C8390","D2678","D0174","D9552","A2177","F1119","C8905","D5254","D6100","A0743","E7184","E9709","C4580","D6186","A1578","C9238","D2042","F2562","D5485","C7469","F0829","D3575","F2403","D6092","E1186","D4956","D9021","E7903","E6912","C3454","D6106","C6256","D1896","F3609","A1208","C4103","C0796","E0687","C8490","D5259","D6599","B9436","A3912","A4663","A4689","A2478","A5785","F2779","A9423","A7056","A6486","A2507","F3026","B5615","A6815","A8086","F3139","A6609","A3036","B5394","A8873","A7580","B1470","B0822","A1159","B2064","B7820","B8373","B6240","B3059","D8861","B7256","A1424","B7851","A9893","C5211","B5750","B5907","B4528","C3969","D0771","D2303","B1110","D0875","B4682","B8354","B5972","C4283","C5700","F2398","F4780","B7875","B2995","E6797","A2230","B2269","A2812","E4808","F3994","C3554","E9962","D9545","F4610","A3642","A9930","F4828","C5769","C5593","B8836","D9405","F3885","C2729","F0692","E0955","E7774","C9858","B9833","D6993","F3927","C1987","B0801","F1632","B0451","F4304","B9116","D7411","E2058","D0570","C9106","F4578","E7367","E1237","E3819","F0119","E8875","F2592","E3172","C1042","D7907","E4318","F4021","A2253","E7027","D0505","D4113","A0485","F2175","D2682","E4921","F1627","E6929","D0955","D8306","D5045","D1489","F1699","C7291","F5377","E8592","F4663","A2727","F5511","D4398","C0610","F3975","F0206","C8520","C7464","D0609","E5557","F4679","F4804","D5528","F3957","F3939","C9737","F1484","E9582","F2357","E9032","C6278","D5164","F4517","D9936","F0881","E3437","F0860","E1787","F1117","C5602","E8259","E8097","A0043","E3365","C4937","E4209","A3458","F0443","E9902","D5673","D5657","D9590","F3815","E4727","C6656","A4834","A4602","A5544","A7142","A6141","A7207","A3175","F3389","B4474","B1376","B8387","B3123","A9103","B1795","B0131","A6959","E2827","B6088","B4518","E9832","B5255","B3409","B4122","A1884","A8959","B7813","F3248","B7279","E2273","E9028","B2700","B7717","D8820","E4245","E3454","B5858","E7222","C2245","D5566","F1523","E0935","E6867","C9029","B2778","C2459","F2571","D5204","F0473","D2010","D9977","F5432","D4955","F4266","D2031","A9731","E2976","C0477","B7649","E1856","D6524","B6581","C4781","B8986","E0942","F1767","D2084","D8936","F1266","E3194","E4768","A2041","C4401","C0308","E0862","F5025","C1298","D5231","E5395","C9393","C5569","D1265","D3165","E3978","F4102","A3519","D5488","E8152","D9208","F4582","F3880","E6973","B9994","F3823","F5047","C2902","F1192","E0717","D5980","E3233","F5226","E3636","F0628","C9837","F1270","F4672","E8848","E8408","F1792","D0853","F1107","D9216","E4567","E8490","E9877","C9549","E8137","C0571","E2060","E1300","D4134","C3520","D3356","C6451","B8678","D0389","D4694","F0487","D5611","D5381","D9821","C9485","E0400","F0223","D7841","C1707","A2875","C9596","D7543","C7988","C3235","F2577","E3696","D9444","E8672","C4533","B8903","E1395","C1530","B9543","F0004","C2306","F1734","E0681","E7944","A1117","A3390","C9845","E5311","D9644","D2155","C4128","E9383","B8811","C6795","F4695","F0818","D3189","F3536","D9840","C4209","A3858","A4380","A4335","A5917","A0591","A5229","A8619","A8941","B6442","A6111","F3468","A7516","F4902","F5316","B1802","B1300","A8611","F3039","B7715","A1347","A6265","B5339","A2169","B7787","F0128","A8083","C0984","B2581","A2648","B0715","F3506","B7662","E1525","B7287","A9290","B3257","A0015","B1473","B7848","E2537","F5048","D4517","B4963","A0866","B7774","B7667","C3231","C2252","F5433","E1688","E7873","E4904","D6969","A1348","E2605","F5168","D9617","D0892","E8345","D9348","B1034","E7601","D3422","E9997","E1663","B0412","C7351","B4629","F4141","C4678","F0913","D1982","E2301","B2360","D8584","A0492","C7559","B7124","C8234","E7636","D5258","D5047","A9030","C0054","C2176","C2528","C6425","D2020","D0151","C5709","B2717","F0668","E7234","D9639","C1148","D3233","F3901","A3561","E3928","F3951","E3101","C5573","D9517","E4767","F0693","D0699","B3557","E7737","B6926","E1281","C6202","D8741","A3598","C8080","F4811","D1958","C4126","A2354","C8210","C1575","D2873","C8964","A2872","C5306","D6942","D9594","D7634","E8538","F3782","C6249","E0387","E6974","C9130","E9320","F2463","C8182","F3860","C5960","D4808","E3871","A2423","E8396","E7316","F4151","F2184","C4999","D7188","E1703","F4301","C0781","D7768","C8013","D2163","C1689","E9889","D6152","E5502","C9931","D7793","D3895","E8039","F2155","B8497","F4287","E7007","F5145","D2118","E1108","F1276","B9523","D0079","E5646","A1640","D1233","C3375","B9352","C5101","B9152","E3047","D4096","B8696","C4554","F1909","E4784","E4509","D6175","A3060","E8089","E5827","E6319","A1089","E2553","A1644","D3653","C7924","C9317","C6869","E1120","C5036","C2406","D4870","D0489","C5177","E7048","C7154","D0730","A0911","F4638","C6657","D9765","C6850","C8785","F0493","C5020","F1894","F1674","E9843","E7427","C7756","F0575","F4731","E9881","C2906","D4145","E6561","C2357","D5479","E0984","C7316","D9991","C3655","D8855","C7128","A0572","F2655","A4415","A4909","A8387","A6002","A5790","A7707","B2675","C7230","A2451","F3206","A8494","B5177","A6893","F4949","B0309","B0501","B4959","B5260","B4121","B4905","B1693","B7966","B7893","A9109","A7103","F5335","B0972","B0471","B4378","B4577","B8913","C8923","B8089","B9941","B6589","A9008","B4724","E3481","B0475","B1140","E2738","E3898","A2871","B7906","B9608","B7883","A9034","C2498","A0635","C8955","B7095","C8267","C3473","A3298","B6942","B0370","D3467","F1765","E4972","D8424","E1550","E3760","F4274","D0560","B8311","D9712","D7091","C1057","B4120","E9713","C5673","E3396","F4677","B4835","D8542","D7341","E7259","F3881","F4507","B5993","D9060","E3131","C9938","D0484","B9089","C3921","A0519","C2118","D0181","C7148","D8157","E2298","E2378","D0065","E0318","C5553","C4945","C7935","F4826","E1613","A2657","E7297","C1580","D9986","D6328","F4745","A0319","C6377","F3834","C2371","C5675","C6399","E1551","E9990","F1997","C7145","C5307","C0133","C2183","F4055","E3623","E7821","A2751","E5358","C2328","C9866","E2476","C8954","D5971","F4363","D0904","E2542","C1155","F1176","E8404","C5006","F4362","A1275","E5904","E1755","C0672","C5727","C1175","A2320","C8350","F2540","E8950","D8513","B8924","F0605","D8540","E2478","E2613","D5583","A1502","A2802","D9244","D7772","C4387","E8286","A0089","C4583","C8584","C5698","F2531","C3416","A1669","C5416","E6990","C2138","D8743","D1486","D3131","C3265","A0199","E7336","D8705","C0429","F2117","E1426","D1960","C5561","A2723","A0366","F1514","C1680","C1299","D6907","B9993","F1036","D0981","C3704","F4198","D3631","F2097","F0759","A2901","D4112","E6666","E4586","C5292","D2449","B9484","B9248","D9988","C2488","E9885","C9213","E4405","E9530","F4608","C1408","F0251","B8743","D9995","D4191","C2630","E0811","D5274","C1414","E0386","E8160","F0614","F2782","A5567","A5425","A5037","F2692","A5887","A5794","A6723","A5511","A7901","A5026","A5264","A7993","A8737","A8904","F2821","A8922","A2498","A8860","B6149","B2337","A7071","B6580","F3301","B3586","B3924","B5323","A1869","B1947","E5238","B5502","B3304","F3319","B3428","B4787","B8076","B4425","E3987","B3175","B7565","B8346","B3995","F3537","C6017","A2248","C5632","B4604","E6735","F4602","A0793","F3830","A1028","C2466","B3717","C3347","A2981","E6814","E7639","E3315","E8777","E9171","C1165","D6986","B0575","F5165","B6594","F4433","E1603","D5757","A2105","E7314","C4219","D4703","B7922","C6137","C8566","D0247","B9577","A9871","B5644","E9496","D5798","E5154","B5953","E2524","F3948","A9301","C0452","C4331","B8984","C7637","D9962","E6966","E5006","E3689","F4239","E3457","C9123","A0431","E0182","C4840","F4300","B8817","D5744","F4031","C3412","C7841","D9982","C3426","F1173","D2100","D1624","E3073","D6568","E1653","E6899","E4857","C5990","B4935","A0481","D4668","D5847","B9027","E7426","D0146","A2186","E5363","F4712","E7893","A1667","E0112","C4060","E3664","D3062","D3050","D5793","E5356","C4165","E6758","F4014","C0846","B9449","E2442","D4305","E8211","C8218","D5716","C9214","D4565","A1649","F1401","D2554","E3975","E8626","D3616","F0451","E7923","E9487","C0668","D3067","D5916","E6732","D3273","F1456","F1035","E8597","D3481","D2928","D8061","D3333","C1977","D6570","E7302","D3451","D0100","E8910","F5454","E5549","E2062","C1194","D7017","D8783","D7811","F1818","D2357","D4552","D1813","D8763","B9584","F4673","C2622","E4829","A3420","C3817","F0660","E0639","C3958","D4212","E4022","E3398","F1698","D7379","E7810","E0730","A2259","E4881","F5252","C0436","C7648","E8850","D2498","F4298","C0418","C4686","F4564","D5831","D3737","D6185","B8865","F0717","D7551","B9443","C1274","C3922","C2370","E9486","D7530","D5863","D9142","E8836","C2897","F2346","D2197","E1798","A2806","E5731","D0029","F1463","A0523","E9276","E3962","D8578","F1291","D6702","C8357","C0227","D1377","E6890","C9364","E5524","D9692","C2754","A3755","A4040","A4357","A4110","A4130","A5932","A7690","A7572","A8077","A7599","A6857","B9962","A3125","A3085","A0595","B6682","B0671","A7393","B6136","A6803","B6517","C1158","A2560","B4088","A6199","A6117","B1587","B6884","B5243","A9065","A1120","E5899","B8007","A6699","B5816","C3966","B5238","F3409","B5544","B3323","F4989","A9053","A9194","A9041","A9360","B0870","F3320","B2871","B0276","B4412","F0584","B0478","B4434","B2317","B3400","B1522","C5555","A9374","B7140","B6537","D0127","B7933","E0132","B0250","B3475","D0917","E0140","D1601","C0372","F5072","F5430","E4422","A0101","A3064","B3419","E5347","E5713","B7781","E2655","D6762","F0104","B1605","D1548","A3288","D8939","B8150","E5830","B0669","F5182","C6552","F5121","F1916","C1700","E6968","A1309","B3219","C6394","C6355","D5306","C6383","E8225","D1163","E5616","F2037","E3820","D7608","B7711","F1224","A1694","B1797","C3424","B7551","C1218","C4833","B9148","B9294","C5265","C1407","C3105","E5945","F1798","A2240","C1994","B9166","D0960","E8356","E0721","A1257","D1224","F1562","C1188","E1494","D3476","D7083","E9710","C0001","E3813","F4666","C3446","C2079","E2715","E8169","D8161","C7112","E8776","A0547","A2367","D5763","C6195","D4301","D0166","D2164","C1771","A1848","E3392","A0410","E7523","D4248","E2878","E3641","E0370","C2742","F2135","D3878","E3473","D9732","C6597","C5951","C5212","B8851","E0529","C1121","D0732","E8854","C2924","F3949","F0336","C2323","E4069","C5384","E7030","F5368","C1412","F0257","C6659","D9238","E6313","E2609","E6467","F3770","A1979","F5515","D9739","E1922","E4102","D4594","E7904","F2379","C0312","F2041","F5258","C0727","B9659","E7305","A3380","C8180","D6548","F4671","C9073","D2629","D2651","E4634","F1081","F4225","C5697","E7145","C0992","A2202","F3546","F2061","E2846","D4580","C3169","C7545","C7152","C3650","C2232","D5395","C5959","D4672","E6321","C3010","D4657","F4753","C7730","D2375","E5309","E7643","D5732","A2321","E7676","D2779","C4513","F1502","E8867","A0999","C3226","D3353","E7146","C2103","A3417","E2507","C4578","C8969","E0076","C1293","E6994","B9164","C5502","D6541","E2154","C5863","C5820","F0207","C3629","D3244","E0980","C9078","D2843","D1060","C2913","D2013","F0831","B8533","F0920","C9053","E1686","A4613","A0201","A4898","A4531","A5603","A4416","A2492","F1014","A7113","F2969","A8407","A7620","E5190","A6966","A6406","A0550","A2996","B7527","A8252","B1264","A6701","B3152","B3448","B2331","B7555","B0986","B1365","E3177","B7630","B1586","B2739","B6800","F3263","A9960","B4344","B2190","A1375","B7767","A9652","B0265","A0734","B6894","B9601","F5346","C9464","C6469","B3237","B8411","B3000","B6769","B4638","B3861","B5257","B4845","A8989","B3042","A9116","B5264","A1426","E8214","D0615","E1317","E8349","A1212","D3174","C8170","C2576","A9592","B9058","B2151","E5454","B0854","E2941","E6661","A2425","C7711","D7845","B4833","B6456","D0831","E5511","B9177","F4543","F3705","B8826","C9439","B0096","F1505","C2041","A8985","A2902","C2301","B4847","D7667","C3056","B0547","A3172","D8977","D7328","C3471","F1937","C6447","F2315","E5062","E6452","D2312","D8439","D6465","D2907","F3538","D6177","C8881","A0448","B2641","D7574","C6218","C6718","B9025","C1716","E6970","E2225","C4249","D7833","C1291","C6380","C5290","C5685","F2339","E1751","E4323","C9308","E4658","F4560","D4938","E6756","B6824","F2188","C1877","E2239","F2452","B8446","D9711","A9955","E5136","D3066","C5708","A9212","C4284","F4680","E8972","C6526","E6823","B3110","C1989","D1867","C7481","D5396","F4377","E0697","D6374","F1852","C1531","F1074","A2238","D0891","D9508","E3633","D4458","D4164","E8311","E7854","B9615","A2089","D7402","D7932","E3400","E3552","C7185","E2928","E9208","E2689","C8692","E1597","A0740","D7096","B8975","D2416","E2787","E2063","D2664","D3352","D3180","E9863","A1991","E2342","C0479","F4440","C6104","E7095","C6048","E1742","C1521","B9730","C4712","F2001","C6072","F5079","F2167","E7567","E4472","E0609","F2104","D9199","C9258","F4676","E2589","D7230","D2694","F4308","C3618","C1628","E0313","D4360","E2243","C7173","D3986","C6696","E3709","C0588","C2985","D8116","D9853","C4164","D8784","F4093","D0175","C3820","E7816","F1333","C1967","E6434","F2331","E8683","E1789","D3607","C7860","D6133","E4866","E6359","F5094","C5734","B9697","C2376","E8229","D6536","B9435","F2218","C3355","A2199","C9112","E3450","C3367","C8645","C7777","D2275","D9140","C8665","E4081","A2096","E1497","C8058","A3500","E8815","E0938","F1402","C0047","E8718","D9041","C2334","D3930","E9284","C7501","C7746","F1369","D9909","E0570","D1173","E7424","C7040","D0205","A1686","C0568","E7235","E4759","D5042","D2645","E5929","E8003","C1819","D3259","D7943","D3026","C3568","E8909","E4604","C0595","E0772","E6917","C2358","D9075","C4032","B9434","F0243","C2253","D2539","E4812","C9353","E9207","A3169","A3762","A3830","A3968","A4079","A4502","A4800","A4904","A4732","A5434","A8109","A1122","F2729","A5889","A7246","A8291","A8289","B4226","A8106","D9619","F2882","B2941","B1189","A4979","A7190","A7623","B1008","A1352","A8343","B5718","B2339","A7158","A7889","B5226","B8086","B0794","B2632","A0423","C7838","B1095","A6373","F3515","B5101","B2505","B0028","B7294","F3491","A9658","B5164","A9980","F1041","B3405","B3352","B6434","B6719","A0472","E9532","F3214","D3059","B3884","A2601","E9550","B6901","F3208","B4991","B3851","B3181","F1191","B5125","B1468","B6098","B6951","F2483","B8405","B4247","B6505","E6882","D1780","E1453","B8149","A1857","F5267","C8323","F1647","F2288","B3495","A1403","C0349","D0254","A7303","D5915","E2710","C8034","C3229","B7531","D4018","F1914","A1350","B9814","D6274","F1585","F2068","F2334","C1052","E7611","F1992","C1419","C1583","E3006","E5140","E6465","C2731","C2915","D9812","C4335","B8933","E5541","E0187","E2084","E0227","C9354","A3197","D2589","E7718","B7502","E1024","B2848","E3917","D9272","F4555","B4723","E6064","F3759","D7376","D8087","C5372","E0918","F0125","C4790","C4250","D5979","D1619","D4843","E0958","E3003","D1194","D3292","E6721","E9809","C7051","E0769","D1288","A3063","A1290","D4003","A2155","E0299","E1113","B7871","B9832","B5890","B8907","B9724","E0245","D7750","C7224","C5346","A0565","D5893","D1935","C1725","E9500","C6682","E0180","E6665","E6446","D6245","D6757","C1763","E4059","A0914","E1051","E5983","C2391","D6520","E3024","D7615","D5176","E9372","C3692","E2001","F0173","E8075","C2345","D8742","A0172","D0188","C2377","D3529","F4763","F0875","F5375","F0835","F4674","D5482","D7319","E9766","C2381","B9416","A2088","C3322","F0689","D0614","B9063","C2241","C8123","C2108","C5824","D4823","C8703","D1115","B9966","E2240","B9142","E1754","C8989","C3273","D2733","E4661","D5026","E1206","D8246","C0241","B9057","B9824","E5121","F0729","E5123","D2500","D7567","C5818","D7326","F2279","C3965","E9906","C0732","B9437","D0551","F1193","F4741","F4713","E8158","E5525","D2025","E1868","D0063","D4690","F4415","E3005","D8089","E4082","E4399","F2414","E6509","E9522","C4220","F2232","C5755","D9581","F0744","E5411","F0370","C7054","F5201","D1086","B9548","E6436","E3372","F0188","D0515","A1164","C3796","D0818","E4425","D5269","A1142","D8729","C9110","D7848","E3720","C9939","E0969","C0671","E9760","C2409","D7079","B7977","E4867","F0306","B9356","E5014","D4646","C7471","B9635","D5938","D4012","E2984","E6741","D8586","C6353","A3643","D9546","E2090","D1772","B8809","B8666","C8056","F4730","E7278","B8772","D6740","D1316","E0045","F0778","D3221","C4579","D1903","F4332","C2551","E1424","D8566","D4152","B9491","C9182","A3032","B9309","D0964","E1909","A4006","A4350","F2717","A0285","A7467","A4902","A1219","A7475","A8727","A6598","B0612","A7120","A3033","A6492","F5486","A3316","A1782","A2493","A7487","A7912","A6639","B0181","B5789","B7427","A6915","B6264","A6542","A8431","B5797","B6143","B7988","D8193","A1682","B7459","B2796","B4036","C9240","B7759","A9572","E8201","B0974","D0094","B4505","E7072","B4896","B0657","B4891","E9732","B7647","F3211","B2503","B5216","D5112","A9325","B6701","B1381","D4856","B6943","D7215","E1736","C0328","B5496","F4457","A2792","B9640","E2219","C3845","D5033","E5600","B8107","E6418","F4035","F0790","B8562","B5237","D9310","D0794","B3390","B7952","C1176","E3307","B1327","F2073","A3338","B4534","D0846","B6129","B8887","C7234","E2085","E8450","F2127","F4528","B8321","E0253","A9334","E1725","D1692","F2426","B0208","E1563","B9477","D3957","D0478","E3714","D4840","A0984","D8277","C9362","D2869","E9535","B9077","E8666","D2257","A2985","E9676","E0172","D3941","D8981","C3155","E0594","F0834","D4515","B4500","C8253","E2480","B9612","E8959","A1025","C8626","D0405","E0328","D7121","C5481","D1653","D7595","E0183","A2287","E7384","C5945","D8616","D9138","D2009","F3879","F0230","E8893","C8079","E4099","E4580","D5212","C1600","D0540","F0374","E9933","E4951","A1706","E4949","C2112","D9597","C8175","C7073","D3332","F2164","E9700","C8857","C7346","E4464","E6877","B9400","C8404","D3057","E0948","F5348","D3477","A2766","E7242","E9040","C0176","C0008","C2462","D4006","D6817","C0267","F4613","F0283","D1129","D7444","D1357","E7486","C5155","B9902","D7904","F5080","C3460","C7978","B9769","F1718","B9955","C6198","D1596","D9392","C4061","E7065","C4541","F1866","E9842","C5163","A0219","F4719","E0453","F1799","E7041","E1434","F4092","E7751","C2840","E1691","D8934","E4063","A2103","E9883","C5320","D4148","E0542","E1979","C3383","D8603","F0520","D6941","D1560","A2109","C8988","E8550","D0058","F4616","D0139","B9564","C7769","C1602","C9581","D9867","D7789","E9361","C0243","C2383","E6818","D9814","E6878","C2182","D1725","F4697","F2030","F4659","E0710","E1443","E1504","D4616","D6196","C0857","E3584","E2652","C1497","E5707","E3257","D9032","E9483","C7823","D9139","F0132","D2679","D8356","E8426","F4162","D1302","E5210","D3201","D0259","D2712","D7645","D0376","D4835","D2537","E1982","C1968","F1347","C2802","D1145","E2615","F3613","D9482","C3724","B9104","D1561","D8758","D3162","E4591","D1623","B9857","E6329","D6308","C8416","E4884","E5024","B9907","E8996","E3569","E8041","F0468","E0825","E6845","D7485","E7501","F1082","D6419","D7298","E5291","E4823","E4919","D0317","C2220","C2303","C3390","C1565","D2600","E1397","E9173","E3363","D8587","E4566","D8006","F5013","A3765","A3846","A4234","A4404","F5287","A5989","A7140","A3177","A5548","A5251","A0908","A1069","A7752","A6467","A7804","A7396","A6145","A6463","A7243","A8096","A6773","B0991","A7882","A7090","B1069","A6275","A7613","A7684","B1654","B7352","B5711","A8880","A3619","A9225","E5027","B6045","E1286","D3695","A9354","F3124","B6460","B6182","B7444","A7085","B3382","B0392","B8341","B4038","B0973","B8209","A9042","B5783","B1380","B1760","A9155","B0407","D6656","B0529","B6640","B6227","D7755","E4929","B6622","B6810","A2666","B1031","B1129","B5809","B5473","A9748","F3313","D3045","B4725","B0745","E5565","F2300","B1664","E3916","D0187","E4688","C8993","E7991","E7689","B4294","B3315","B5997","A9369","C3320","B5019","B7097","F3046","B6586","D2687","F4979","B7325","B6472","E7177","D8408","E8365","A9471","B0289","D9600","E2365","D1378","E3919","E4968","B8313","A9240","D2270","C9061","B1643","B2417","C0335","E9646","D1376","F3267","C1302","C6121","B7746","C6935","E6102","D4519","C0697","F5376","D6916","A1993","F2416","B7370","B3618","E4279","E7605","F0166","E8479","C4030","D0900","A2149","D8263","F4614","E0243","C0946","D9288","B9634","D9873","E1952","E3564","C7209","D9303","A1593","C3644","C2825","C8392","D4804","D2427","E0015","E8033","E4762","E5432","C1706","C2259","E5224","C8944","D6529","E8290","A2136","A3663","B1234","C2980","E3790","E1367","D3919","D7583","C9306","C5591","D2662","F0901","E7022","E3269","F4278","C6057","E9981","F2407","E1843","D2940","F0622","A1925","E7410","A2276","F4454","D3324","D5217","E0349","D1256","D2987","C1346","E3585","E2608","D9835","E5985","D3492","F0194","A0870","D4696","D2446","E8324","E0007","A9383","D5587","D8913","C4723","C7389","D9456","D1865","D9847","D8493","C7131","E1322","D6973","E2332","C1439","E2749","D0132","A1647","E9418","D5303","C4756","F0478","A2966","C9868","A1079","C4468","D8175","E8856","E8896","F4349","D8849","C7923","D4588","D3293","C5250","A3065","C5733","D1344","D6489","F0033","C8709","E8861","C5165","E0729","D1492","D5853","D6544","E6157","E0059","F4460","C7133","C6575","D3852","D0397","C8035","E9466","D3226","C3048","D9595","D9147","D3851","F4360","B8744","A2061","D7176","D5213","D4724","A1741","E2244","C5548","E8190","E3366","F5577","D4619","F5222","C5350","E1161","F1307","D1991","D6241","C1064","D5685","C2974","D1518","C8120","E4380","A1029","C1109","D9507","C3703","E8156","C6045","D9192","B9805","A3543","C4156","C7267","B9678","C2235","D8296","C9730","E8062","C8646","B9976","E4128","E3665","E3174","E4847","E3509","A1707","C0289","E7046","D1153","D1660","E8125","E7196","E9594","F0683","D4722","C0963","D6678","F0227","E1236","E5026","F2099","C6731","C1219","E6557","C6826","F1098","E0738","B8925","E2841","E5083","D3931","A1643","E1749","C1492","D5930","E1450","D3557","D1924","F0253","C7713","C3705","D3758","B9315","D9632","E9988","C6306","E3756","F2067","F5391","C8346","E5193","E4924","F1179","C8719","D4525","D4487","D4346","D2881","E2445","F3958","E7332","E2234","D1878","F2190","B8916","E5298","E9919","F4618","C4965","D2507","F0615","F2316","E7372","F1421","D1245","E0310","E3495","D2655","D2683","E5552","F0906","D6189","B9873","D5702","F2442","B8895","D6711","C4129","C2250","F4626","E6664","A3510","C7662","A3424","C4466","A3807","A4290","A4397","A4612","A4595","A5415","A4752","A5880","A5539","A5793","A7763","A7788","A5085","A8275","A5432","B6692","A1454","A8379","B1546","A1240","A5077","A7644","A6308","A8059","A6612","B0857","A7652","F1773","A0184","A8820","A2637","B1172","B1896","B1749","B2285","B2081","B1690","A9077","B7418","B7012","B4756","B1740","B5263","B8316","B0655","A7108","A6900","A8638","B5656","E6045","B6661","E3060","E7887","B5894","F4954","B4089","B7525","E9021","A9730","B1275","A9009","D5348","A9688","E0422","B3274","B6752","A9096","B2785","B5186","B6421","F3607","D3682","F4224","A8689","E8191","B3534","C9429","D7187","A0704","B4491","D5500","B1296","B0328","B7255","C2660","F3765","D6543","E6426","E3300","B0074","C7962","E5958","C7776","B5481","E9655","B2943","B2831","C2081","B0275","D1208","B0713","C1593","B2771","B1631","B5742","C2887","C1425","C2098","A2644","F0890","E2211","E1010","F4372","E2890","E7419","B0377","D5003","B2994","B0331","E4752","C9446","E0139","B8858","C6403","E4007","C8640","C0973","E6036","C9839","D8812","C5480","E0615","D7740","D9207","C4179","C7667","D4757","C2322","C6832","E2759","C9830","C2170","D6305","C5015","E5883","D0134","C5770","D0821","F2189","A9079","D2840","B9837","D0470","D1945","A1134","B9062","D2591","D8739","C1665","D7985","A2413","C0738","E5040","B8208","B8892","E1800","E0225","D7978","E5188","B0913","D4713","C7077","B1939","D5307","C3415","F4606","D5497","E4159","E5161","D8070","E1231","C9842","F1114","C6180","D9529","A2189","D0820","C3261","B8943","E6085","E3446","A1969","E7682","B9056","D3749","E4865","E6745","D5262","B9257","F4776","D3224","D2431","E5367","C7952","B9617","B9071","E4948","F4265","E4693","C3964","D7700","B9076","E0822","D8250","E3020","B1212","E7955","E9921","D9518","C1520","B8771","C0087","F5188","C8523","C4438","E0169","F4152","F1273","D1379","C8223","E0438","C2161","F4490","E1244","B9969","C6777","E4664","E5745","D6354","D6077","E5337","C6883","D4029","D4656","C7526","E2395","E6153","D9947","C2020","D5185","D6535","E0789","C7287","D2001","C8922","C8720","D9415","C9358","E6924","D0647","E0879","C8983","D1349","F0447","C7238","E0047","C4162","D1140","D8418","C5878","D0843","C8791","C2677","F2133","F1883","E7362","D4940","D3232","C2741","F2573","E7825","D0194","A2198","C2281","A1472","E6144","A1596","D0931","E3433","F4153","F3750","E3843","E4048","F3887","C6342","E5866","C6331","D3078","E6538","F4121","E0364","F0139","E4538","E7630","E2019","D8102","C9718","F4472","C4715","E9602","E7657","C5157","D7947","F0210","C6207","E6842","F1714","C4710","A3200","C0612","A1785","D5533","D3956","C3275","D2698","D6369","D5442","D9960","D6412","E8536","C7674","D2590","E8025","C9523","C4035","B8982","C7955","C3153","C2558","C0618","D7994","F4270","E7738","D1337","C0675","E9381","C5487","D6232","E1928","D7677","E4127","E7656","C2820","E9890","E5276","F0872","E9367","C6371","F5078","D9965","E1681","C6189","C2651","D4213","D5129","D0836","B9136","D1142","A2205","C7292","C8705","D7823","B9467","C2052","C0235","E4263","A2094","D4401","E0790","E7766","D0236","C2344","C3293","E5693","E6203","F0641","D9547","E6437","C7492","F5177","D8832","E9025","C2885","C1204","D2534","E2678","C7284","E2355","D3969","F1113","E0835","D1380","E7528","A3370","C3144","A3747","A4495","A4590","A4501","A6048","A0609","A5123","A5676","A5230","A5098","A5792","A7642","A8306","A7922","A5561","A5106","A5502","B0411","F2860","B6279","A6794","B7162","A6318","A8276","B6629","B2896","A7688","A6779","B2711","A8215","B6857","A7532","C4888","A9001","B5346","F2861","A1864","B2874","B2707","B2892","B9119","A9888","B0975","B7447","C1668","B0536","D0464","A1511","A9121","D8225","B1977","B2807","A7680","B4538","A7602","A9093","A9293","B0027","A6310","B7802","B3758","D5351","B0011","B3417","B0217","E7513","B4823","E1079","B4875","C1053","F4365","B7992","B5962","D4249","B8369","A0344","F5306","B1948","C7075","B1107","B0864","C8808","B4710","E8264","B1867","A0052","B1419","C7833","B6223","B1475","B4958","B5363","B2546","E9436","B6345","B0678","C2962","B3721","B0361","C8161","E5963","B8116","C4667","B2069","C3643","D3859","A9329","B0220","B5590","E0231","E4260","D5366","D8969","A8969","C9860","C0961","F2121","B7765","C5050","D0138","C8952","B9972","E6438","B5388","E8832","E9780","F1429","D0525","A0202","A9043","B8878","E9538","D3437","C2929","B3811","E3793","E2782","B3864","E2880","F4100","F2399","E8482","E9678","B6040","D7878","A9751","D6822","D5207","A2589","A8979","E6093","D2715","F5217","F4166","D0901","C6482","C4303","A2651","C6079","E7686","E6408","C9174","C3879","D7641","E8472","E3925","B8386","E7104","C6645","E1334","E4713","C3741","C7630","D8183","D8514","A1940","C5286","E5888","A2211","D1731","D2906","A0669","E3748","E2207","C8412","C6361","D5778","E5049","C8982","E2659","D2404","F4408","C1574","B0097","D1763","E6503","C5297","D0572","D6255","E2895","D6937","B8894","D7977","B9609","E2411","C5839","F5045","E7182","D7636","D9933","C6523","C3819","A9919","A9024","D6712","D3274","C3961","E7033","D0997","C5692","C4277","E1989","D9340","D2719","F0528","D7390","E3688","E5287","E9764","C7020","E3182","B4040","F0354","D5906","E2245","B8869","E3681","E7150","F3172","C0399","E4852","B6603","D3750","D7646","B9004","E3249","D9449","C3863","E3217","C6555","E6657","C2789","E5673","C5359","D1281","E7441","E4200","D7465","E7723","E1006","C2728","E3468","F4446","C8229","C0050","D0208","C3920","B7714","D5020","A3587","C4135","E2843","D7867","E5443","E6551","E6626","D0483","C4636","C9027","A0446","D1773","C0343","F0357","F3664","D7380","D7338","E2942","E8675","E4655","C7937","C9767","D6501","D7552","E8685","E7602","E2173","E1803","C3907","D3721","C5568","E8933","C3827","F0680","D2154","D1818","C5470","C9370","D0380","D1401","F0508","C5611","E6082","E4536","D1270","E5114","E5551","A1466","F4611","C7815","F2122","E4231","E3877","D9214","E1293","E0347","D6062","D2029","A2887","F0154","C0397","E6607","D7179","C7421","A2759","B8589","E6207","A2242","E8874","D9412","D6729","D5896","C4718","D2352","C5584","F0097","C4005","B8690","E3674","E5566","D9540","B9626","B9371","C3038","D7107","E2933","F2533","F1089","F0849","C6721","C4588","A0180","D5187","C4203","D9704","D1766","A2146","D7102","C3497","C3116","E4406","F1902","D6226","F4583","F1513","F0571","D5156","E5398","D4943","D6352","A3589","D7884","F1930","E5654","B9879","F4261","D4304","E4446","D1723","F4524","B9550","A0396","D4941","C9043","B1828","A2359","E2993","E1230","C7009","F0346","E5761","C9932","F5567","D4476","D2149","C5936","E7233","D5297","E9661","E8064","D4345","F1490","D9690","C2368","E9140","D0329","C0705","E5726","E4612","C1309","A1652","D9270","C1343","D9998","F1292","D3809","C1740","D0731","E4533","B8732","D4652","D9344","D8644","D7720","C3343","D6425","D8944","D9837","C5440","C5472","E8881","E3082","B8709","C8071","C2586","E0392","A3925","A3863","A4242","A2437","A0739","A4646","F2724","A4935","A5972","B4596","D5054","A5550","A4949","A6968","A0697","B5827","A1197","A5057","B0104","A7548","A6658","A6534","A8272","F2832","B5428","A7550","B2525","B2837","B1297","D7138","A8473","E3485","B5204","B1076","A7833","B1864","A7163","A7803","C9281","A7342","A9783","B5678","B4955","B8090","B3412","B1224","B5414","B8976","A6085","B7102","B8439","C3381","E7475","B3986","B6165","E6062","B7844","B3118","B1041","B3739","B5405","C5447","B9752","B5126","A9733","A9367","B2643","A9241","F4361","A9003","D4740","B7171","B5516","B3389","B1153","F5262","A9094","E5115","B0912","B1656","E0494","B3199","E8273","B1021","B2764","B4193","E0375","B5717","D3934","B3011","C4043","C7699","F2047","B7993","B5877","B2307","B5603","C9808","E8126","D5619","B2478","B8898","D4666","B2843","B5310","C0864","C9571","B8347","E2174","E2292","E2944","C5870","F4237","B6190","B5133","E1573","F0510","B1695","B7901","E2529","E3512","F1625","D8065","C2284","E8283","F1112","E8665","C5437","B7522","D8047","E0913","B3027","D6020","C7804","F1679","C9083","E8148","A0031","C9451","C2444","A2232","D1921","E8326","D8412","D0621","F1319","C6173","D4177","D5737","C8334","C4400","E0923","D8750","E7767","D3961","D9380","D4618","C9765","D6037","C3822","E4627","D0762","D9905","F2328","E1965","F2456","D5165","E0411","E4711","D6513","D7596","D0834","C1943","C7528","E1295","D7919","C0099","F1598","F2222","D8836","C7119","C1612","C7100","D0591","D3655","B8883","B4863","C8286","D3965","F4155","E1793","E4805","C8264","D4174","F2581","F1120","C4295","C7219","F4541","C8589","C4663","F0495","A9823","D8518","E3767","E1562","F0137","E1645","D3187","D3029","C9871","C8333","E8540","C4591","E5954","C6540","C5768","E3753","E1328","C5479","D3056","E7219","E5266","C5023","F3931","C3181","E4467","E7823","D9850","C8154","E4503","C9176","B0434","F0327","D5807","B8679","F4090","E5371","C8230","B9274","E4218","E4589","E1403","C9898","D9955","D2383","E4532","E2106","F4421","E7271","E5037","E4037","C6854","D3080","F0365","C5607","C9256","A2868","E1053","F1768","D9521","E2994","F4782","E1659","D7297","A2025","F2023","C2049","C9896","E5714","F0270","D7643","E8731","D9599","F1747","D0754","D8005","E7163","B9429","D3900","B9987","E9450","C5226","D2837","F5345","C7741","C5869","C4880","D5322","E2950","D1290","E9479","C6819","E3570","C8418","D8315","A0165","A0761","D1306","F0209","C6660","E0359","A2225","D7408","E4773","C6844","D1575","E0883","D5250","E5592","E2297","A1123","C2223","E9382","C7466","E6208","D9946","D8286","D2966","D7253","C7565","C1484","F3707","C0565","C5245","C9272","A1097","D4739","F1631","B9988","E9464","D5239","E5812","D3106","C3368","E1330","D3245","C7221","E0619","C5952","C6650","F0919","D8199","C3918","D8105","E7844","E2420","E4119","D7725","E9309","E7068","C6364","C5866","E8001","D2239","C1267","E0185","C9831","C0390","F3653","C5494","E0689","E4249","F1237","E1914","E6749","D2665","F0490","C3374","D4065","D9501","C0691","D2821","D4316","D6844","D2603","C8913","D2506","C3637","D1201","F1379","D6573","D2833","E2661","E1400","C4236","C2115","B9384","D9789","C8436","C8362","C0100","C2795","D3242","C1253","D4147","D6644","C5495","C8534","E7617","D8744","D9523","C2201","A0072","D8805","C8076","C4088","C2195","D8432","D3328","C7280","D9790","E9258","C4074","C1216","D1348","E1528","C7666","C8781","F0844","A0182","A3999","A3989","A4034","A4146","A0732","A4653","A4779","A4519","F2758","A6929","A5103","A5653","A6922","A5732","A5509","A8258","A7117","A7876","A6553","A3315","A8843","A8160","A7789","A9114","B0278","A8708","A7461","B1320","B1229","A7881","B3073","D8013","B8224","B5887","A7799","A2019","E2147","D0939","B2353","B1879","B4831","B3647","E8355","C4430","B2335","F1527","C1093","B3121","B4216","A0778","F3474","B7334","B8396","B4887","B6187","D2123","A9728","B7553","B7218","F0126","A9977","B1108","B0609","B5434","B1149","A0088","B3057","B5399","B0381","F0674","D2090","B1048","A1170","B1329","B5638","B0092","F0522","B3429","B7400","D7084","B6339","B0144","D5037","B5982","D1916","A3307","D3648","E9781","D2787","B4788","C5694","A9784","E8886","B9819","E3110","B9247","B4176","C3621","C2658","E1314","A0006","D8628","B2292","B3780","B0930","B6799","C9649","B7762","D7315","B6493","E5446","E6837","F4466","A9111","E3376","F4474","E2654","C9552","E5747","D1241","B1151","C0577","D7809","E2767","D3486","A1088","D1184","D2686","F1668","E9812","E2492","A3110","B6282","E3749","C7954","E2412","E9567","F1242","D3515","D9505","B7668","E5104","E0266","E0445","F4656","B2816","E8615","C8942","E2625","D0024","C3578","C7220","E3062","E8171","C8237","B5548","C3242","E0252","E4291","C6058","C7673","E3344","D0626","E1809","C0223","E5216","A9760","E5438","E0904","D4725","C9836","D5133","E8447","C7771","D0170","D7227","F1096","C2454","E7540","B3746","E4328","F0501","E6317","C2891","C6210","E2422","C3699","E0320","F3329","C4680","D0833","C0402","E2939","F4385","E9159","C9210","E9605","C1246","C9374","D8569","B8881","C9105","E4113","D9479","E4494","C9935","E9471","C4867","B9801","B8807","C4065","A3291","A2111","D8119","C4642","D4020","E0335","D3289","D5545","E0093","D1209","D6697","D2266","E5272","E1418","B8580","F4690","C6390","C4281","C1625","E4809","D2736","C5980","D9262","D8834","E2453","D0634","C7339","E6011","E8682","E0839","E4869","D8173","D4448","E7998","D8039","E8453","C7635","C3919","D6560","D7899","F1739","C3768","D1390","D0941","D3926","D6659","B9768","E5619","C2399","E4351","D3210","B7724","E9512","C7443","D0557","F5144","D9358","F2130","B8622","C2127","C9229","D7796","A1727","E4096","F1875","C9442","B8559","C2414","D7877","D8302","F1759","E0033","C4681","C4538","C3999","E1576","F1341","C1928","D4114","F5159","D7974","C6397","E4224","E0481","E7564","F4766","D8583","E1937","E5877","C5910","D8112","E2731","E1165","D5144","D4136","E9574","D7716","E2065","E6190","C8653","D9439","A1689","E0451","C8718","D3951","B9959","C1996","B9656","C8036","E0518","C6831","C7948","B9312","E8361","E6327","D7508","D9537","C4497","E6242","D3789","E8321","D9673","B9110","F1428","D7022","C7921","D9918","E7586","F1483","C0105","C2280","D2670","E3514","E6485","E7099","F0941","D9502","C9269","E7109","D5410","B8862","F2086","D2003","E6201","C0274","C3623","C8461","E0994","D6484","D7477","D2811","E1476","D9651","D0243","E0150","E5657","D5505","C1940","E7898","D6525","C1207","B9282","E1156","D4654","B9569","D5220","D2871","C7701","C8552","C1963","E6219","E8596","E2061","D2636","D1727","E1093","E0358","E5704","A4305","A4777","A4801","A4255","A5111","A4617","A4434","A0004","A4901","B3692","A6019","A5738","A5913","A5108","A5289","A7505","A5862","B7563","A5043","A6573","A3550","A8304","A6764","A8782","B0528","B2897","A9213","E6508","A6498","A8488","A8770","A6732","B3606","A9267","B7945","A7826","B0202","B2491","B1776","B4846","B2703","B4467","C4892","B8015","B5514","E6618","B3912","B0717","B4053","B3404","B4743","B3101","A9704","B5699","E7085","B3330","B5305","B0797","B3795","A9884","E1808","B5036","B7718","E6074","B6361","B2001","B4973","E9410","D6348","D5761","B4652","B5272","D9001","B9127","B5058","B1805","C0782","E2875","D6078","B6955","D1225","B5584","B1226","B7382","B5839","A9118","D2857","C6736","F2079","B7061","E0872","F3622","A9938","B6512","A0860","B3348","B1694","D2796","D5779","F2385","B1968","E4121","B3441","A0692","B1518","E5109","C6297","B5171","F5381","D3976","B0895","E8066","A1628","E6301","C8774","B7052","C1545","A1982","C4699","D3921","E7996","E9811","D0971","C1672","E5258","C9922","E6857","A9939","C6214","D2367","B6053","C3305","B0454","C3178","C7816","D8004","D9870","D4347","D5311","D2813","E8240","C7899","B5266","F5480","C4028","E1121","E3432","A1558","C7498","B8384","D0907","B8871","D8595","C6175","E9544","C7638","D9028","E5627","D4575","C3587","D2667","F2559","C5896","B5771","E7199","C7965","C8588","E4794","D3646","E1229","C3545","C5394","C6538","D3763","E4344","D5145","E3817","C1178","A1267","E5158","D6239","A2064","E6893","F0607","E0674","C4600","E3266","C0386","C9222","D3113","C8846","D6405","B9255","A2339","C3745","C6263","E6088","E2471","C3114","C5410","C3067","E6881","C9862","D1640","C6423","D0826","D0431","E0235","E9037","E4626","C4726","E0136","D5312","D6496","C6343","C9584","D0678","B9581","D3299","D0280","E0959","D3091","D4188","D4198","E4926","E9399","F1715","E5620","B9545","C9157","D7775","F4484","D6167","D1351","B9351","F3910","B9412","D5331","E9925","C9510","D5290","C6245","D9413","C2008","F1614","D0890","C7682","D1609","C7684","C3729","C3713","D2292","B8713","B8746","E3690","A1978","F5260","D5705","D2101","D7832","D4196","E3111","D5612","E8735","C4792","E1358","E2828","D3182","D3181","C1874","C0916","E4373","D3036","E5223","D7347","C1709","C5932","C9564","D5840","D2005","C2950","E8795","E4647","D8431","D0461","D0010","E9478","D1198","D6706","E1783","B8802","D2956","E9782","C3914","D3630","B8734","E9932","D0263","B9298","E2168","E4515","C4139","D6200","D8504","A1828","D7349","A3426","C6759","D6784","F1763","F4342","D5618","E2201","E6534","D2480","F5357","D9073","B9500","D5564","C8391","E2813","C2965","C2760","D8510","F4683","D4907","D3466","C9887","C3764","D9325","F3630","D9924","F4240","C9098","C2879","C1058","D1071","F1983","D9560","E1008","D4023","E8011","E9200","C8009","F5041","D7198","E3990","E4198","C9982","E3448","D4916","F2420","C5144","A2159","C0551","C6664","D5371","C4989","F1026","D5270","C5564","C8619","C2955","B8759","C5385","C9410","E1534","D1309","D7456","A1569","E5289","C8500","E5050","E3162","E8301","D1605","B8677","C4354","C1200","B8682","C3725","C7099","E7369","C2609","B9385","D3715","C4581","C9934","D2447","B9386","F3847","E2554","C7198","C5466","E0333","E0628","A3844","A4159","A5546","A4219","A4496","A5089","A4848","A5455","A8980","A6733","B4330","A7067","B1670","A6196","A6599","B7326","A8076","A6653","A6510","B2349","B2354","A8442","A7683","A8062","A1074","B2840","A7502","B4772","F5445","A7232","A1428","A8401","A7214","A6414","B6370","A0074","C7547","B6221","E2910","B3767","A7866","D0893","B4421","B6587","B5262","B4948","B4185","B0525","C7523","B7646","B7544","B0449","B3030","B6288","B0461","B9731","B1995","F3258","F4629","A2970","C8278","B3564","B7089","B3839","E6366","B3117","F3236","B3214","B5119","B1050","E7275","B4341","B2398","B6991","B1676","C1378","E3511","E3314","B5689","B2852","B0541","D3540","F4041","F4838","B5577","E0530","B3894","D4180","B2044","B2165","F2255","E6424","B8142","B5952","B7683","C2707","B6324","B0429","B6320","C8963","B9269","B7879","E4229","C8984","D8233","D0857","B4314","F4334","C0804","F0163","F4772","B3242","E5717","D8462","D0806","B3827","B1241","B5785","D9331","D3499","B6687","B9570","B5763","B2385","E8227","F5060","B4084","E3540","B9765","D1683","D9903","B3217","C1917","E5025","B5040","C8978","D5109","C0395","E7386","B6631","B7997","C7235","E5630","F1283","C9952","B8551","F1797","F2386","F4036","C2966","F0603","C5409","E7277","F1735","B5547","B5140","B0807","E3750","C1649","B6529","B9439","D1454","F0120","C5281","C2090","C1449","D3478","C9601","D5459","C4565","B4260","D2676","D7041","E2098","E6779","C8714","C2036","C1655","D4911","F5193","B9970","D9797","B9733","D5172","A3330","C5807","D7859","E7012","E8559","E4196","F4497","B5289","D7850","B9811","F4106","E3812","E0004","C7134","D2951","C4751","A1802","D1317","C0221","F4447","E4089","C1181","E9972","C1004","B9179","E0196","D2730","E0603","C1567","D2594","D0035","E2707","E7801","C0638","C8933","A1701","D2277","B9174","E4003","D3436","C4623","F1350","F0885","E4717","E4336","F4273","C2151","C4977","C9780","E2326","E6112","E3055","D0935","D0152","D2191","C4314","C2954","E8135","C4748","E5036","E8070","A1399","C3957","B3207","D2181","D4019","E1627","D9481","E5445","E9674","D0805","E1574","B9246","D2296","E3207","E9499","D5268","C2267","D4281","E2940","C2903","E8551","D6626","B9735","C2572","B9263","E1324","E6897","B9519","C6346","C7521","C1430","C7691","D6326","D5973","C2113","E8509","A0452","E2757","E5428","B8653","E8207","D5458","D7269","E6323","A0191","C0327","D9396","C5441","C5223","D4857","C0714","B8687","E5313","E6368","F0134","D7563","C4208","D5714","B9983","D8400","D6012","D0084","B9295","B9876","C8650","D5345","D3136","B8967","C7554","B9061","C8540","D6505","C5846","E0258","C6565","C1137","D5805","D9971","C3838","E4967","D4460","F4109","E6141","F4320","C0614","E2180","C8301","E4846","E7733","E7498","E2350","C0716","F4757","D9700","A1674","F1512","C9877","E7310","C2645","F2333","D5529","E8531","D6635","E1160","E5530","E9505","C9732","C7025","E6256","D9335","F3564","C7702","D2846","E3390","C4238","C3989","C3791","D0073","D4782","C5186","D4397","D2872","E7433","E5405","D9204","E0009","D1758","B8751","B8915","D5816","B8661","C6904","D9602","C3350","D8711","E4819","C6413","C2321","D7190","C9814","B9496","F1725","E1915","E3108","E5508","C1872","E4551","D3525","D6864","C3616","C6700","E0309","B9954","C0598","D6539","E1310","D3813","E3853","D7794","E4563","C2266","A3915","A4210","A4240","A3218","A5315","A5328","A5395","A8547","A7847","A7280","A5720","A6299","A8605","B5398","B7310","A7187","A7940","F2923","A8164","A7122","A8412","F3514","A7535","B3183","A6269","D4033","A9281","B4722","B1589","B7987","B5546","B2985","D7117","B2481","E0011","C0091","B2568","B3439","B4447","B0439","A7755","E7692","B0514","B3637","B6526","B2716","B4773","B1759","A7581","B4022","E7140","B8342","B6658","B4888","B1374","D7043","E1571","B7388","B1496","B7385","B0179","D1501","B4321","A2592","A9157","B4546","A9389","A9887","A0408","E8483","A8977","B0172","B0147","B0463","B0263","A9247","E4153","B3045","C3795","E8458","D7577","D2510","C7966","F3404","B0358","E7628","B0419","D5135","E7229","C4558","C1440","C5550","C7687","C6114","C3297","D6478","C1390","C9107","B5031","D4216","A0309","B9729","E2318","D8620","C9616","E5477","C9757","E2679","D2886","C7822","E3048","C0065","D7912","E1445","D0499","B4132","D9787","F0381","D8600","C0603","D0648","C3224","E5424","F5451","D4800","C9425","C5455","D6295","C3058","C7279","E7599","E5554","E2534","A1579","B8576","D2762","D7701","E2160","C3453","F1572","D3053","B0238","E1878","D1470","B8796","E2178","C2580","F1210","C2627","D6768","E7217","C8610","E3605","E1807","E9323","A0747","E5472","C4658","E3581","E2300","C4254","D4592","A3652","A1076","D8261","F0195","D3769","C8526","E3329","E3991","B9086","E1274","C0817","C9507","F3913","C4972","C0231","E6810","E8532","E5638","D4000","C8792","C0096","D9606","E4985","C5213","D7606","E0827","E6870","E4830","D3857","C2968","C2704","C7473","C3470","D0925","E3057","D6736","E4364","A2330","F2351","E6240","E0636","E4475","D5375","D6800","D1190","E1380","D8060","A3489","E8602","D1643","D8480","D4810","D3995","E9589","C7811","C7694","E4681","D2986","C2851","E8415","C5326","E6608","B3139","C6398","E6137","D2002","C1592","F4096","E2159","E7134","E8123","D2891","D3280","B8825","D2039","D3534","D6051","C0341","D6257","A2319","D2203","D4561","C2798","A1692","E3214","C7368","E9551","B9515","E3286","E0686","F1011","D2768","D9319","D3150","D4331","C8444","D4160","C7189","E4167","F1038","E3474","E4726","D3071","B9316","D1526","D4076","B9878","B9245","C3489","C0654","E2927","D4340","D9793","C4163","E7050","E7360","E8571","B8737","E8691","E2002","C1596","E5846","E6119","C0238","D0477","A2297","E1092","E5912","D0005","F0888","F4104","E1248","C7717","C6948","D5698","C0382","F0323","D9414","D8135","B8776","D3568","D4799","A3743","A3749","A4262","B0399","A3023","A5027","C1773","B8404","A6552","A7891","B4166","A6702","B7006","F3006","A7170","A6784","A8274","E0303","A5233","A1949","E8309","A8760","B7797","C9437","A8570","B4250","B7199","A0841","A7820","A7894","B3492","B3660","A7849","F5255","B7029","B0078","B7454","B3814","B1590","A1059","B8083","B0337","A9278","A7352","B4667","B3843","B7323","B1958","A9395","A9456","B4729","D8188","A9450","B8254","D8887","B6376","C4168","A7682","B4249","D3999","B5760","D3384","C0807","B6300","B4311","B3750","B0697","B3869","F0940","D2285","B7079","B4037","B6899","F1182","D2428","B3332","D4049","E9643","F1325","B0484","D1509","B1998","C8973","E8756","B5578","E5206","E3827","C1614","A0422","C2561","B5867","E6549","E5884","D2442","B8765","D3351","B4864","F2555","D6911","E1240","D9328","E6061","B9478","B0842","E8093","D4567","A8999","A1834","A9859","A2219","E1988","D2231","D8020","C5947","C8609","D6073","C6077","D3977","D5967","E7721","C4574","C9565","C2287","B8712","C0936","B3010","B6811","B3824","E7105","B2322","D9621","D7571","C3735","D6347","D5754","C6792","D3751","D2703","B1979","D8989","C7550","E4689","C4282","B9687","D2763","D4545","C8638","C3951","D9659","D4510","C8160","D0054","E8963","B4561","A1998","C3213","D5353","C3508","C7909","C9219","D6938","D6144","C9770","F4227","D2410","D7351","C7907","E6239","B8663","E2421","E8802","D4214","D8167","D5168","A1533","C2222","C0904","C7311","B9107","E0694","E0648","E0034","D1948","E8235","D3115","C4874","E4571","C0419","C5227","C0409","F4464","C1005","C8023","C9488","D7630","C3307","E3881","C2379","E9756","E5327","E9430","D2011","D9202","E8454","E6411","D1077","C1622","C8060","C7591","C5145","C7974","D7310","C8093","E5068","E8102","C2607","D7318","C4981","C9785","C9066","D0679","D3341","B9498","E3328","C0251","D5800","C2823","E3536","D1250","D7203","D9015","C8932","D4576","C6842","E3183","D6142","E5046","A0097","D9336","E5157","C8022","F4229","B9065","C8630","D0906","C2800","C4444","E7407","D5869","C6120","D9162","C9583","B9501","E5732","E8705","C3925","E8868","C8826","E8168","E3635","E9896","D3839","B9134","F0015","E7941","D6035","E1466","E7710","D9653","A1467","D0313","D8865","D8007","E1858","E9209","C0229","D9418","C9350","A1272","C3187","D3679","C2417","B9403","D6606","A1215","F1726","D3846","E9148","C7182","C4800","C5018","E5663","D2176","D3554","E9958","D3784","E0902","B8868","D9898","D9897","D7270","D4073","D3640","F4518","F1415","F2261","E2556","E3774","C1258","D4202","C0528","C6360","F4381","C9602","A4212","A4264","A4866","A4538","A5601","A5562","A5453","A7622","A5866","A7724","B4885","B7139","F3370","B4275","A6998","A6431","B2704","A6965","A8891","B7090","B5746","E4078","B3077","B4469","A7529","A8310","F4509","D3096","B2801","B0236","B7018","B2661","A6339","B4521","B0107","B2827","B5610","B0145","C2960","E3519","D0324","B4819","E6247","B7258","B6109","B7692","A9169","B4750","B6089","B6193","B9549","B6151","B0803","A0849","D6526","D1049","B3376","D4640","E7799","E1088","C0841","C3813","D2571","A7307","E2526","C2588","B0259","B5144","C3841","B5534","B0559","F0108","B4000","E7144","D6821","D2770","E2000","B2539","D1137","E9107","B0390","D6162","A9185","A9678","D9019","E8323","B1591","C0348","C6873","D7593","B2117","D4054","E1541","D6395","B9715","E2997","B9066","D1485","E8103","C3657","D6153","D9943","D7267","D3194","C9211","B3784","E5471","C3756","D3360","C9242","E8108","D9780","E7485","D2153","D6829","C8155","C5194","E1414","C0026","E4569","D5710","E4018","D3060","C6673","D3728","E7855","E7660","E6790","E0074","B4347","E2354","E0741","E8042","C4903","B9713","F0634","D9605","D8726","D7021","C5679","E6860","C7934","D4599","C6680","C2050","E2406","F2252","E6244","B8633","C8712","D7452","B8745","B1135","D1794","D7125","E6405","C7205","D7124","E8964","C7513","A1948","C7632","C5511","E0535","E1177","C6699","D3906","D3947","C2262","E4216","D4465","B2981","D1218","C6841","D0297","E9591","F4784","C6287","C5982","D1227","E0771","C4746","C9063","E4477","D7714","E0815","E7680","D0425","B9998","D5822","C9479","E7508","C5560","E9263","D6422","C0676","E2826","D0897","D7998","F5489","F0408","E1983","D6394","D2213","E0649","C4496","C8304","C7427","B9306","D2082","C4819","C2123","E8771","D2301","D3854","D3643","E7654","C9046","E3211","E9942","E1181","E0611","D4676","D8745","B8592","E6419","D7196","C3216","E0154","D9483","D0051","C3294","C1677","D4635","E4418","F0925","D9957","D9796","C4056","E2050","C9434","D4734","C4409","D6201","E2496","D6947","E6947","E2458","C1131","D6349","C6250","A3481","C6259","E2853","E9518","B4481","A4160","A5973","A4577","B1485","A5259","A5726","A4925","A6827","A6631","E8608","B2649","A8558","B0910","A6994","B3615","A7174","A9387","A7837","A2409","B0725","A8019","A6249","B3002","B6747","B2604","B0248","D0932","B1689","D4055","E5617","B7297","E9514","B3349","B2727","B3081","B0251","B3177","B0341","C2571","B6261","E0035","B6524","B0152","F4452","C3833","E2143","C9426","D9691","B5270","F2025","D4845","B6215","B4202","C7429","B0694","B4114","A9997","C5623","D5359","B2089","F0176","F3152","E5964","E6971","C9249","C4685","E9573","B9117","D6886","E5719","E6076","B5602","B2996","D9818","B1086","C9531","D0476","C6339","D8910","B3707","D3741","C6572","E3920","B2936","B6096","B9098","C5773","C0656","D4532","D9327","D3790","B2310","C4185","C0489","E1638","D9172","D8831","D9999","D5741","D2812","B4848","D1995","C7613","C5780","B8615","C1442","E1222","D3950","C9784","C7603","E8185","C6642","D1933","B8566","B4067","B9083","E4130","E2111","E2629","B7518","B4857","D7941","D8878","E4354","E7377","D6066","D4010","C7653","D4753","E1001","E2199","E2026","C4493","C0154","E4065","B8572","A1693","D4175","E9792","B6831","E5736","E9882","D5422","C7076","E8695","D2402","E0632","E6717","D9980","C0125","D6143","C5486","D7123","C4361","D5550","E5845","E9783","E0161","D0991","D5169","D5295","C1973","E5475","E6572","D2027","E9950","C7250","C6778","D5326","E7667","D9125","D2105","B8534","D2391","A1534","B9289","C1922","E0987","D1444","E4690","E7588","E0233","C7901","A0094","D2878","D0089","E5381","D8402","B9226","B9343","E0288","E1232","D6674","D9135","C1341","B8535","C2324","C1571","C0725","E6794","B9427","D7032","C4363","E2362","C4534","C4625","D1052","C0430","E2681","F4412","C1254","C0383","D4527","F4779","D2262","C1902","E4210","E7901","E1867","E2740","D9347","D4437","D4632","E8194","E3477","C2217","E1441","D2017","E2717","D3141","C9753","A4082","A4226","A4426","A4703","B4830","A8070","A6284","A8264","C0830","A7483","A1173","B2189","B1899","B3358","A8154","A7315","A7507","B7600","A9453","A7888","B4606","B2578","B8234","B4700","B1148","B1316","B7954","E6624","E5204","B2761","A0100","B6035","D9092","C8325","A6750","B0180","D5937","F3274","C1160","C5216","E1277","B6334","D2946","D3473","E0322","E5261","B5945","B1359","D8473","A9397","E3839","B3609","E8155","C3995","C8231","B0269","B9132","B9683","D5814","C9253","A8998","C9283","C4069","C1184","E1630","C2457","D8241","E0796","B9578","D6335","B8715","D6356","E6364","D9253","E3428","B8870","D2654","B9670","E1729","C8618","E4000","B3120","C3732","E4253","E1312","C6407","E0372","D3877","C6010","E1515","C5703","E1545","B5539","B9573","D3739","E0348","E9642","A3397","F4503","F4444","B9842","D7201","F4026","D8613","C8822","E0454","D1730","E4851","B9822","E6613","F2269","C8443","D2862","D2994","A3199","D1300","B9085","D8875","A1698","D9871","C3983","C4383","A1575","E7875","F2058","E2604","D7366","B8680","E8951","D0294","D8637","E5692","A0029","E0693","E1700","B8645","D6510","E3431","D7042","C0928","E8421","C2096","C1881","D1785","D7640","F0328","A2922","E7029","D2836","A0176","D2470","D8601","E9474","F3599","D1061","A4508","A4803","B7227","A8337","A6560","A5648","A8906","A6137","A7350","B7141","A9441","C4877","D8898","A6407","B1901","E9273","E6472","B7125","A8972","A7217","B6659","B1900","E0100","D3419","B2086","B4764","B4792","D6002","B1067","B1203","B4797","B6572","D2012","B1582","B5916","B3565","B2799","B7557","B3347","B5910","E4259","B6590","C7524","B1592","E0018","E1643","C4085","E4349","C9746","D5098","E1517","E8149","C8753","E2172","B9348","F4346","D5095","B0467","D1298","C1142","D0987","C4340","B4286","F2043","C0549","C4064","E9081","D2688","D0135","E0425","D2505","B9256","C2100","D8043","C0112","C1793","E3903","D8670","E7141","A3399","B8598","E1056","C4370","D0407","E3520","E2030","C3358","C8797","E9286","C5948","D6043","D1674","D7598","C8651","D0919","D0296","E1266","E2932","E5031","F0661","C8956","C9993","E2248","E0446","C0566","C7170","F2162","C3190","A1714","E1124","D9387","C8701","D7580","D6423","D8032","E2293","D9677","C8007","C4670","D1588","C6853","B9984","C3264","A1621","D2199","C2237","F1823","B9505","D3267","E6255","B8516","C1522","A1525","C3359","E4255","E1423","F4245","E6855","D2148","E6303","A0398","C6894","D8967","E7274","D7174","E0830","C4539","E0985","B9367","E6614","C6576","A5815","A8255","B0589","A8185","A7890","A9522","A7435","A6727","B4246","B1941","B4829","B6407","B3794","A6795","B1299","B8233","B0395","E4480","A0429","B2890","A8391","B2053","A9808","A9624","B6396","B8365","B5017","F5192","B1888","B7497","C2128","B1347","C7043","B1559","E4993","B6535","D5334","D1919","D3163","E1812","C9411","D3745","E5112","E7071","C0709","D2293","D8701","C0800","B5775","D2529","E0475","B0720","D5785","D2988","C3938","F4095","E2183","C6369","D4505","B8605","D3693","E5848","D9492","D3335","E3442","D1328","A0234","E3986","B9100","E2594","C5821","D6672","C7639","C9376","C4652","C1541","C0098","B8578","E8529","B8991","F4767","E6008","E7568","C2374","C7122","B9281","D7908","C9235","D4092","D8196","D5170","E1258","F0025","F0604","C3540","E1179","B8735","E3030","D7792","E3362","D6538","C1743","E7015","D8194","D7746","C7762","E0993","E5335","D5273","A3527","E4953","D5333","C9287","A2435","A5138","A5616","A6409","A5113","B6387","B2028","A6868","B4990","A7492","A7924","B5579","F2477","A9759","B5376","B3374","B2726","B7118","B1972","A9089","A8726","B4733","B2314","C2434","B3314","A9675","B6686","C9514","B4765","D2041","B8261","B1266","D8507","C3492","F0537","F2072","C6556","E4086","E4416","E3952","D1322","E3007","E6505","D9290","E5480","F5167","D3736","E6599","C3576","B8575","D8796","E0914","E4352","C5181","D3684","E3799","C7403","D6534","B9693","D1516","E8799","E2739","E0964","F0113","D9472","C3099","D2234","E1632","E9431","C4755","E0711","E2559","E4878","D7223","D3729","F1019","D5167","D3621","E2824","E4918","A0521","E9886","A2133","E1385","C9091","E9829","D2839","E5715","D1954","D5722","E2403","E2320","E5918","B9934","C7397","D0657","D7031","F4693","A5189","A6326","C1808","A6907","B0710","A8591","E9790","B0627","D2450","F4405","A9657","D3372","B0086","B2352","F3422","B3318","B1134","B7327","E2303","B0759","E1863","D8186","D5572","C8141","C8173","E6158","D0566","E4481","C8279","E8556","D1936","A2463","F5465","E8494","C6254","E2664","A2386","C3295","C1585","C9255","E2946","C8820","D5531","E8552","D8997","E5639","C0752","B9506","D4254","E7405","E5343","E9297","E0321","E8838","C9609","E5738","E3297","D7579","B9499","C8004","C2947","C8693","B8920","D9101","E0002","C5775","C4017","C2788","D0578","A5873","A4726","B5075","A6556","A6351","A7269","A6519","B7001","B6669","B4465","B0452","B4758","B5134","A9711","B0271","A9625","B1895","B5293","B2616","B7338","B1632","D3732","B1431","B5654","D7787","B1144","A2072","E6808","B4566","E7502","D7056","C9251","E8399","C5147","C9291","E1253","C9926","C2532","E8373","E4345","C9906","E6506","C4647","C8272","D2316","B0396","E5633","F1880","C2858","B8564","E7943","E8641","E9748","C1801","D8275","E5873","D2081","C2362","E7358","C2160","D1314","E2960","D1872","D8497","D3206","B9732","A5566","B6501","E2548","B1642","C1837","C6782","F4007","C3409","A9990","E2431","E8844","B0650","E3745","B1635","D7776","C5001","A9845","C6030","D1717","C2045","E1872","C5572","C8684","C6372","C0421","E4152","D2456","C4967","E6037","D3860","D9815","E1247","F4103","E6285","D0744","F1424","E5703","E9960","C9221","B9286","E1366","A4585","A7148","B2638","A7526","A7800","C7618","B5888","D9150","E4468","E6449","E1971","E7537","D3291","D3345","C8213","D6597","D0209","C9972","C5776","E7992","B5517","B1721","A6233","E7062","B1535","C7525","C8288","B8221","C6511","C0253","D9880","C7980","E5124","C1503","E9784","C0533","D7395","A8099","B2678","C2758","C9962","B6291","D9498","C6893","B4350","D8237","E7216","C0808","D8806","D2962","E8804","A0414","F0631","E5435","C2565","A4211","A2784","A6897","C3506","E3784","B2137","E6869","A2304","B6591","E7267","E1321","F2760","C9521","D1185","E3265","D8488","D8484","E4505","D2413","B2765","C8280","E3998","D5342","D3731","D0905","D5339","B3379","F2906","A6738","E8028","C2708","C0103","E9529","A1242","C6649","E2519","F2186","A4594","B6607","D6965","F3998","B7425","F3031","D4763","C9075","A0538","F2143","F1944","A6215","B5374","B1609","B5794","E9734","F4504","B5609","A2940","F5404","B9514","C2749","E6619","D4560","C6483","A8191","F0029","C4235","A2031","C6793","E9023","C9264","A6025","E9160","C2158","D5912","D8770","B1053","D1910"]

In [7]:
import requests
import pandas as pd
c = ['cetno','name','v_cat','Rank','clg','g_cat']
df = pd.DataFrame(columns=c)

for i in range(0,len(c_l)):
    try:
        url = 'https://keaonline.karnataka.gov.in/pgcet_result_2024/main/resultscheck.php'  # Replace with your URL
        payload = {
            'txtrollno':c_l[i]
        }


        response = requests.post(url, data=payload)

        response.text



        import bs4
        htm = bs4.BeautifulSoup(response.text,'html.parser')
        l = htm.body.find_all('table')[2].find_all('table')[1].text.replace(" ",'').split('\n')
        lst = list(filter(lambda x: (x != '' and x != '\r'),l))


        df.loc[i,c[0]] = lst[3]
        df.loc[i,c[1]] = lst[6]
        df.loc[i,c[2]] = lst[9]
        df.loc[i,c[3]] = int(lst[11].split('-')[-1][:-1])
        df.loc[i,c[4]] = lst[16]
        df.loc[i,c[5]] = lst[22]
        print(i)
        if i % 100 == 0:
            df.to_csv('mca2.csv')
    except:
        print('unable to fetch ',c_l[i])
    


0
1
2
3
4
5
unable to fetch  F4981
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
unable to fetch  F4945
33
34
35
36
37
38
39
40
41
unable to fetch  B9831
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
unable to fetch  E8154
61
62
63
64
65
66
67
68
69
70
71
72
unable to fetch  F4366
74
75
76
77
78
79
unable to fetch  E5975
81
82
83
84
85
86
87
88
unable to fetch  F3884
90
91
92
unable to fetch  A1151
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
unable to fetch  E9364
121
122
123
124
125
126
127
128
129
130
131
unable to fetch  F3754
133
134
135
136
unable to fetch  F5305
138
139
140
141
142
143
144
145
146
147
unable to fetch  C0192
149
150
151
152
153
154
155
156
157
158
unable to fetch  F2987
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
unable to fetch  F4627
180
181
182
183
184
unable to fetch  D1597
186
187
188
189
190
191
192
193
unable to fetch  A4316
195
196
197
198
1

In [ ]:
import requests
import pandas as pd 
from bs4 import BeautifulSoup as BS 
import json

# defining the data frame where we have to store the Data from WEB
col = ["CET_no",'Name','Rank','Verfied_cat','Cat_alloted','College_alloted','course','SNAO','Fees']
df = pd.DataFrame(columns=col)

# loading  PGCET numbers from JSON file
with open('pgect_no.json','r') as  file:
    data = json.load(file)

# fetching all the PGCET Numbers
pgcet_numbers_list = data.get('pgect_no',[])


# fetchinng all the data from the data from website through url
start_number = 0 # this is just to start from that last number featched to csv file if something happens while the loop is running

for number in range(start_number, len(pgcet_numbers_list)):
    try:
        url = 'https://keaonline.karnataka.gov.in/pgcet_result_2024/main/resultscheck.php'  # Replace with your URL
        payload = {
            'txtrollno':pgcet_numbers_list[number]
        }
        response = requests.post(url, data=payload)
        res_text = response.text
        
        # convertint text to readable HTML using Beautifull soup
        HTML_text = BS(res_text,'html.parser')
        lst_1 = HTML_text.body.find_all('table')[2].find_all('table')[1].text.split("\n")
        lst = list(filter(lambda x: (x != '' and x != '\r'),lst_1))
        
        df.loc[number,col[0]] = lst[4].strip() # CET_no
        df.loc[number,col[1]] = lst[7].strip() # Name
        df.loc[number,col[2]] = int(lst[13].strip()[:-1].split('-')[-1].strip()) # Rank
        df.loc[number,col[3]] = lst[10].strip() # Verfied_cat
        df.loc[number,col[4]] = lst[24].strip() # Cat_alloted
        df.loc[number,col[5]] = lst[18].strip() # College_alloted
        df.loc[number,col[6]] = lst[15].strip() #course
        df.loc[number,col[7]] = int(lst[27].strip()) # SNAO(Serial Number of the Allotted Option)
        df.loc[number,col[8]] = lst[-1].strip() # Fees
        if number % 100 == 0:
            print("updated to CSV upto", number, pgcet_numbers_list[number])
            df.to_csv("MCA.csv")
    except:
        print("unable to fetch : ",pgcet_numbers_list[number])
df.to_csv("MCA.csv")
